In [1]:
# Script to Clean, Tokenize and Ngramize Downloaded articles
# Steps:
# 1. Import libraries, make list of csv's in directory
# define clean function (turns raw OCR into cleaned lists of tokens) <- problems: OCR-errors (buitenlandsche)

In [2]:
import csv
import os
import glob
import pandas as pd
import re
import nltk
import webbrowser
from nltk.corpus import stopwords
from collections import Counter 
from nltk import ngrams

In [3]:
os.chdir('/home/ruben/Downloads')
csv_filenames = glob.glob('*.csv')

In [4]:
def Clean(string):
    strip_special_chars = re.compile("[^A-Za-z]")
    string = re.sub(strip_special_chars," ", string)
    string = string.split()
    string = [w.lower() for w in string]
    #string = [word for word in string if len(word) > 1]
    #string = [word for word in string if word not in stopwords.words("dutch")]
    return(string)

In [5]:
df = pd.DataFrame()
list_newspaper_titles = list()

for file in csv_filenames:
    tmp = pd.read_csv(file, sep = "\t")
    print(tmp.paper_title[0] + ": " + str(len(tmp)) + " articles imported")
    list_newspaper_titles.append(tmp.paper_title[0])
    df = df.append(tmp, ignore_index = True)


Utrechtsche courant: 23627 articles imported
Provinciale Overĳsselsche en Zwolsche courant : staats-, handels-, nieuws- en advertentieblad: 47201 articles imported
Bredasche courant: 47353 articles imported


In [30]:
def GetBigramDf(input_df, clean, n):
    bigram_list = list()
    
    for i in range(0,len(input_df) -1):
        bigrams = ngrams(Clean(input_df.ocr[i]), 2)
        for g in bigrams:
            g = ' '.join(word for word in g)
            d = df.date[i]
            d = d[0:4]
            tmp = g + " " + d
            bigram_list.append(tmp)
    
    op = pd.DataFrame(bigram_list)
    op.columns = ['bigram']
    op = pd.DataFrame(op.bigram.str.split(' ',2).tolist(), columns = ['w1','w2','year'])
    op['count'] = 1
    
    if clean == 'yes': 
        op[~op.w1.isin(stopwords.words("dutch"))]
        op[~op.w2.isin(stopwords.words("dutch"))]
    #    #dfo = dfo[len(dfo['word1']) > 2]
    #    #dfo = dfo[len(dfo['word2']) > 2]
    
    op['bigram'] = op[['w1', 'w2']].apply(lambda x: ' '.join(x), axis=1)
    op = op.drop(['w1','w2'], axis = 1)
    
    return(op)

In [31]:
df = pd.read_csv(csv_filenames[0], sep = "\t")

In [ ]:
op = GetBigramDf(df, "yes",2)
op
